In [3]:
# imports
from bs4 import BeautifulSoup as BS
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


In [4]:
# set plotting params
plt.rcParams['font.serif'] = 'Ubuntu'
plt.rcParams['font.monospace'] = 'Ubuntu Mono'
plt.rcParams['font.size'] = 18
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['axes.titlesize'] = 18
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['legend.fontsize'] = 18
plt.rcParams['figure.titlesize'] = 24

In [5]:
file_path = '../data/ufodata.json'

In [35]:
reports = []
with open(file_path) as f:
    for i in f:
        reports.append(json.loads(i))
            
print(len(reports))

136494


In [9]:
class CleanUFOs:
    def __init__(self, file_path):
        self.file_path = file_path
        self.rows = []
        self.load_data()
        
    def load_data(self):
        self.reports = []
        with open(self.file_path) as f:
            for i in f:
                self.reports.append(json.loads(i))
                
    def parse_html(self, report):
        try:
            bs = BS(report['html'], 'html.parser')

            text = str(bs.find('tbody'))
            text_lst = text.split(' : ')
            occured = text_lst[1]
            occured = occured.split('  ')[0].strip()

            body = text_lst[2]
            body_lst = body.split(': ')

            reported = body_lst[1]
            reported = reported.split('<')[0].strip()

            location = body_lst[3]
            location = location.split('<')[0]

            city, state = location.split(', ')

            shape = body_lst[4].split('<')[0].strip()

            body = body_lst[4].split(':')

            duration = body[1]
            duration = duration.split('<')[0]

            description = text_lst[-1].split('">')[3].split('</font>')[0].replace('<br/>', '').strip()

            data_dct = {'occured': occured, 'reported': reported, 'city': city, 'state': state,
                        'shape': shape, 'duration': duration, 'description': description}

            self.rows.append(data_dct)
        except:
            pass
    
    def clean_reports(self):
        for report in self.reports:
            self.parse_html(report)
    
    def to_pandas(self):
        return pd.DataFrame(self.rows)

In [10]:
clean_ufos = CleanUFOs(file_path)

In [11]:
clean_ufos.clean_reports()
df = clean_ufos.to_pandas()

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000


In [12]:
df

,occured,reported,city,state,shape,duration,description
0,5/6/2017 05:00,5/6/2017 4:10:01 AM 04:10,Camp McGregor,NM,Light,10 minutes,Light seen over mountain's east of Camp McGreg...
1,5/6/2017 04:50,5/6/2017 5:00:54 AM 05:00,Mojave (Canada),BC,Light,1,Light in sky stationary. Not a airplane or an...
2,5/5/2017 11:30,5/5/2017 12:18:44 PM 12:18,Austin,TX,Disk,3 seconds,"Flying saucer descends, possibly lands in Nort..."
3,5/5/2017 03:00,5/5/2017 3:49:05 AM 03:49,El Mirage,AZ,Circle,30 seconds,"While letting my dog out, a very bright white ..."
4,5/4/2017 23:34,5/4/2017 10:38:52 PM 22:38,York,NE,Fireball,0,A fire ball was moving in the atmosphere while...
...,...,...,...,...,...,...,...
131980,5/15/1982 20:00,12/8/1996 01:45,West Reno (out on old Hwy 40),NV,Other,1 minute,"SUMMARY: Loud boom, so loud shook house. Wen..."
131981,8/15/1979 22:10,12/17/1996 13:50,Beach Haven Terrace (Long Beach Island),NJ,Fireball,4-5 seconds,SUMMARY: Fireball flying parellel with the ho...
131982,7/1/1972 19:00,12/10/1996 19:21,Provo (south of),UT,Disk,20-30 min.,"SUMMARY: 1972, Utah-small bright cloud, I'm d..."
131983,7/15/1965 16:00,1/28/1997 17:27,Nyona Lake,IN,Cigar,30 minutes,"SUMMARY: Cigar shaped object. 25,000 to 40,00..."


In [13]:
df.shape

(131985, 7)

In [44]:
import nltk
import string
import os

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer

In [23]:
def tokenize(text):
    tokens = nltk.word_tokenize(text)
    stems = []
    for item in tokens:
        stems.append(PorterStemmer().stem(item))
    return stems

In [25]:
token_dict = {}

for i, desc in enumerate(df['description']):
    token_dict[i] = desc.lower()

In [27]:
tfidf = TfidfVectorizer(tokenizer=tokenize, stop_words='english')
tfs = tfidf.fit_transform(token_dict.values())

/Users/josiah/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'thi', 'thu', 'togeth', 'twelv', 'twenti', 'veri', 'wa', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_w

In [32]:
response = tfidf.transform(df['description'].tolist())
print(response)

  (0, 206200)	0.11262785114065649
  (0, 205337)	0.136798258349096
  (0, 203886)	0.05537274294357027
  (0, 201672)	0.23172542787218797
  (0, 181847)	0.16208057301982728
  (0, 179812)	0.15416020758069815
  (0, 171181)	0.1082210124371745
  (0, 167719)	0.09738861279412166
  (0, 160170)	0.12903187920404105
  (0, 148075)	0.09796818732254856
  (0, 138269)	0.09706078728925804
  (0, 137572)	0.09470426618400542
  (0, 130866)	0.1909131039500867
  (0, 124875)	0.47261630799618226
  (0, 121272)	0.07696334523730819
  (0, 119434)	0.08337565518385594
  (0, 117925)	0.12526184585273017
  (0, 104010)	0.13185112649406333
  (0, 101500)	0.19104649334398435
  (0, 100533)	0.15738246110185747
  (0, 100382)	0.21001803501714716
  (0, 79032)	0.1208445801032912
  (0, 73728)	0.11132096356023233
  (0, 64282)	0.24721761690766522
  (0, 55816)	0.24761317400113678
  :	:
  (131984, 187418)	0.21997045494429465
  (131984, 175906)	0.3979188256255913
  (131984, 172934)	0.13391057313538413
  (131984, 169205)	0.0677010940863970

In [33]:
tfidf.vocabulary_

{'light': 117925,
 'seen': 167719,
 'mountain': 130866,
 "'s": 1122,
 'east': 79032,
 'camp': 55816,
 'mcgregor': 124875,
 '.': 2795,
 'hover': 104010,
 'spot': 179812,
 'look': 121272,
 'like': 119434,
 'helicopt': 100533,
 'wa': 203886,
 'way': 205337,
 'bright': 52754,
 'went': 206200,
 'higher': 101500,
 'height': 100382,
 'stay': 181847,
 'disappear': 73728,
 '(': 1537,
 'nuforc': 138269,
 'note': 137572,
 ':': 32301,
 'report': 160170,
 'appear': 40595,
 'consist': 64282,
 'sight': 171181,
 'venu': 201672,
 'pd': 148075,
 ')': 1538,
 'sky': 173661,
 'stationari': 181680,
 'airplan': 36192,
 'ani': 39353,
 'known': 114337,
 'star': 180890,
 'flicker': 89843,
 'possibl': 153207,
 'sourc': 177426,
 'elect': 80306,
 'remain': 159787,
 'anonym': 39526,
 ';': 32334,
 'provid': 154892,
 'contact': 64465,
 'inform': 108307,
 'citi': 60021,
 '``': 32493,
 'mojav': 129417,
 "''": 11,
 'bc': 47244,
 ',': 1642,
 'canada': 55987,
 'fli': 89835,
 'saucer': 165134,
 'descend': 71114,
 'land': 1

In [40]:
tfidf._count_vocab(df['description'].tolist(), tfidf.fixed_vocabulary_)

({'light': 0,
  'seen': 1,
  'mountain': 2,
  "'s": 3,
  'east': 4,
  'camp': 5,
  'mcgregor': 6,
  '.': 7,
  'hover': 8,
  'spot': 9,
  'look': 10,
  'like': 11,
  'helicopt': 12,
  'wa': 13,
  'way': 14,
  'bright': 15,
  'went': 16,
  'higher': 17,
  'height': 18,
  'stay': 19,
  'disappear': 20,
  '(': 21,
  'nuforc': 22,
  'note': 23,
  ':': 24,
  'report': 25,
  'appear': 26,
  'consist': 27,
  'sight': 28,
  'venu': 29,
  'pd': 30,
  ')': 31,
  'sky': 32,
  'stationari': 33,
  'airplan': 34,
  'ani': 35,
  'known': 36,
  'star': 37,
  'flicker': 38,
  'possibl': 39,
  'sourc': 40,
  'elect': 41,
  'remain': 42,
  'anonym': 43,
  ';': 44,
  'provid': 45,
  'contact': 46,
  'inform': 47,
  'citi': 48,
  '``': 49,
  'mojav': 50,
  "''": 51,
  'bc': 52,
  ',': 53,
  'canada': 54,
  'fli': 55,
  'saucer': 56,
  'descend': 57,
  'land': 58,
  'north': 59,
  'austin': 60,
  'natur': 61,
  'trailat': 62,
  'approxim': 63,
  '11:30': 64,
  'balconi': 65,
  'apart': 66,
  'saw': 67,
  'ob

In [50]:
from numpy.linalg import svd

In [45]:
tf = CountVectorizer()
document_tf_matrix = tf.fit_transform(df['description']).todense()

In [47]:
panda_ = pd.DataFrame(document_tf_matrix)
panda_.columns=tf.get_feature_names()

In [51]:
U, sigma, VT = svd(panda_)

In [52]:
import plotly.graph_objects as go

In [ ]:
fig = go.Figure(data=go.Choropleth(
    locations=df['state'], # Spatial coordinates
    z = df['total exports'].astype(float), # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Reds',
    colorbar_title = "Millions USD",
))

fig.update_layout(
    title_text = '2011 US Agriculture Exports by State',
    geo_scope='usa', # limite map scope to USA
)

fig.show()

In [49]:
df['state'].value_counts()

CA    14893
       8339
FL     7668
WA     6115
TX     5415
      ...  
DC       27
PE       24
NT       22
YK        6
Ca        1
Name: state, Length: 70, dtype: int64


In [57]:
sorted(df['state'].unique())

['',
 '.',
 'AB',
 'AK',
 'AL',
 'AR',
 'AZ',
 'BC',
 'CA',
 'CO',
 'CT',
 'Ca',
 'DC',
 'DE',
 'FL',
 'GA',
 'HI',
 'IA',
 'ID',
 'IL',
 'IN',
 'KS',
 'KY',
 'LA',
 'MA',
 'MB',
 'MD',
 'ME',
 'MI',
 'MN',
 'MO',
 'MS',
 'MT',
 'NB',
 'NC',
 'ND',
 'NE',
 'NF',
 'NH',
 'NJ',
 'NM',
 'NS',
 'NT',
 'NV',
 'NY',
 'OH',
 'OK',
 'ON',
 'OR',
 'PA',
 'PE',
 'PQ',
 'PR',
 'QC',
 'RI',
 'SA',
 'SC',
 'SD',
 'SK',
 'TN',
 'TX',
 'UT',
 'VA',
 'VT',
 'WA',
 'WI',
 'WV',
 'WY',
 'YK',
 'YT']

In [ ]:
states = ['', '.', 'AB', 'AK', 'AL', 'AR', 'AZ', 'BC', 'CA', 'CO',
          'CT', 'Ca', 'DC', 'DE', 'FL', 'GA', 'HI', 'IA', 'ID', 'IL',
          'IN', 'KS', 'KY', 'LA', 'MA', 'MB', 'MD', 'ME', 'MI', 'MN',
          'MO', 'MS', 'MT', 'NB', 'NC', 'ND', 'NE', 'NF', 'NH', 'NJ',
          'NM', 'NS', 'NT', 'NV', 'NY', 'OH', 'OK', 'ON', 'OR', 'PA',
          'PE', 'PQ', 'PR', 'QC', 'RI', 'SA', 'SC', 'SD', 'SK', 'TN',
          'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY', 'YK', 'YT']